In [2]:
GENE_SELECTED = "DIS3L2"


In [3]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import concurrent.futures
sns.set(style="whitegrid")

In [4]:
map_file = '../data/Achilles_guide_map.csv'
data_file = '../data/20Q4v2_Achilles_gene_effect.csv'
map = pd.read_csv(map_file)
achilles = pd.read_csv(data_file)

Let's see the files

In [5]:
map.head(2)

,sgrna,genome_alignment,gene,n_alignments
0,AAAAAAATCCAGCAATGCAG,chr10_110964620_+,SHOC2 (8036),1
1,AAAAAACCCGTAGATAGCCT,chr12_95003615_+,NDUFA12 (55967),1


In [6]:
map.shape

(74362, 4)

In [7]:
achilles.head(2)

,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000004,0.181332,0.089101,-0.193867,-0.024587,0.038458,-0.181824,0.351672,-0.440073,0.292582,...,-0.124218,-0.469306,NaN,NaN,0.257361,0.244834,-0.408512,0.284734,0.226412,-0.149566
1,ACH-000005,-0.076383,0.245190,0.191238,0.153008,-0.197035,-0.323295,0.252522,-0.571498,-0.068945,...,-0.212442,-0.426151,-0.068295,0.031635,0.205469,-0.068536,-0.092858,0.074640,0.028779,-0.264840


In [8]:
achilles.shape

(808, 18120)

#### Some preprocessing

In [9]:
achilles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Columns: 18120 entries, DepMap_ID to ZZZ3 (26009)
dtypes: float64(18119), object(1)
memory usage: 111.7+ MB


In [10]:
achilles = achilles.dropna()

In [11]:

achilles.columns = achilles.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True)

In [12]:
achilles.head(2)

,DepMap_ID,A1BG_1,A1CF_29974,A2M_2,A2ML1_144568,A3GALT2_127550,A4GALT_53947,A4GNT_51146,AAAS_8086,AACS_65985,...,ZWILCH_55055,ZWINT_11130,ZXDA_7789,ZXDB_158586,ZXDC_79364,ZYG11A_440590,ZYG11B_79699,ZYX_7791,ZZEF1_23140,ZZZ3_26009
1,ACH-000005,-0.076383,0.245190,0.191238,0.153008,-0.197035,-0.323295,0.252522,-0.571498,-0.068945,...,-0.212442,-0.426151,-0.068295,0.031635,0.205469,-0.068536,-0.092858,0.074640,0.028779,-0.264840
2,ACH-000007,0.102195,0.092449,-0.045926,0.171892,0.140561,0.170971,0.086060,-0.432320,0.010664,...,-0.083183,-0.269196,0.101146,0.277820,0.208814,0.001393,-0.327514,0.048714,-0.372854,-0.433157


In [13]:
achilles.columns = achilles.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True)
achilles_clean= pd.melt(achilles, id_vars=["DepMap_ID"], var_name="genes", value_name="value_gene_selected")
achilles_clean.head()

,DepMap_ID,genes,value_gene_selected
0,ACH-000005,A1BG_1,-0.076383
1,ACH-000007,A1BG_1,0.102195
2,ACH-000009,A1BG_1,0.142342
3,ACH-000011,A1BG_1,0.280082
4,ACH-000012,A1BG_1,0.033135


#### Maybe store the number in other column? 

In [14]:
genes = pd.Series(achilles_clean.genes.unique())
genes[genes.str.startswith(GENE_SELECTED)]


4125    DIS3L2_129563
dtype: object

In [15]:
achilles_clean[achilles_clean.genes == 'DICER1_23405']

,DepMap_ID,genes,value_gene_selected
3196802,ACH-000005,DICER1_23405,-1.342802
3196803,ACH-000007,DICER1_23405,-0.993394
3196804,ACH-000009,DICER1_23405,-0.409258
3196805,ACH-000011,DICER1_23405,-0.792070
3196806,ACH-000012,DICER1_23405,-0.786011
...,...,...,...
3197575,ACH-002463,DICER1_23405,-0.959784
3197576,ACH-002464,DICER1_23405,-1.603509
3197577,ACH-002467,DICER1_23405,-0.777286
3197578,ACH-002508,DICER1_23405,-1.030868


In [16]:
achilles_clean[['gene', 'gene_id']] = achilles_clean['genes'].str.split('_', expand=True)


In [17]:
achilles_clean.drop(columns=['genes'], inplace = True)
achilles_clean.head()

,DepMap_ID,value_gene_selected,gene,gene_id
0,ACH-000005,-0.076383,A1BG,1
1,ACH-000007,0.102195,A1BG,1
2,ACH-000009,0.142342,A1BG,1
3,ACH-000011,0.280082,A1BG,1
4,ACH-000012,0.033135,A1BG,1


In [18]:
achilles_clean.shape

(14096582, 4)

In [19]:
map[['chromosome', 'avg_pos','_']] = map['genome_alignment'].str.split('_', expand=True)
map[['gene', 'gene_id']] = map['gene'].str.split(' ', expand=True)
map.gene_id = map.gene_id.str.replace("[()]", "", regex=True)
map.head()

,sgrna,genome_alignment,gene,n_alignments,chromosome,avg_pos,_,gene_id
0,AAAAAAATCCAGCAATGCAG,chr10_110964620_+,SHOC2,1,chr10,110964620,+,8036
1,AAAAAACCCGTAGATAGCCT,chr12_95003615_+,NDUFA12,1,chr12,95003615,+,55967
2,AAAAAAGAAGAAAAAACCAG,chr4_75970356_-,SDAD1,1,chr4,75970356,-,55153
3,AAAAAAGCTCAAGAAGGAGG,chr2_33588446_-,FAM98A,1,chr2,33588446,-,25940
4,AAAAAAGGCTGTAAAAGCGT,chr19_19891600_+,ZNF253,1,chr19,19891600,+,56242


#### Let's compute the correlations

In [20]:
achilles_clean.head()

,DepMap_ID,value_gene_selected,gene,gene_id
0,ACH-000005,-0.076383,A1BG,1
1,ACH-000007,0.102195,A1BG,1
2,ACH-000009,0.142342,A1BG,1
3,ACH-000011,0.280082,A1BG,1
4,ACH-000012,0.033135,A1BG,1


In [21]:
data = achilles_clean.copy()
selected_gene_data = data[data['gene'] == GENE_SELECTED]['value_gene_selected']

# Create an empty dictionary to store correlations
correlations = []

gene_count = 0
def calculate_correlation(selected_gene, other_gene_data):
    correlation, pvalue = pearsonr(selected_gene, other_gene_data)
    return correlation, pvalue, -np.log10(pvalue)


# Get unique genes in the dataset
unique_genes = data['gene'].unique()

correlations = []

# correlations for a single gene
def calculate_gene_correlations(other_gene):
    other_gene_data = data[data['gene'] == other_gene]['value_gene_selected']
    correlation, pvalue, logpvalue = calculate_correlation(selected_gene_data, other_gene_data)
    return {'SELECTED_GENE': GENE_SELECTED, 'EVALUATED_GENE': other_gene, 'CORRELATION': correlation, 'PVALUE': pvalue, 'LOGPVALUE': logpvalue}

# Use concurrent.futures to parallelize the computation
with concurrent.futures.ProcessPoolExecutor() as executor:
    for gene_count, result in enumerate(executor.map(calculate_gene_correlations, unique_genes), 1):
        correlations.append(result)
        
        # Print progress every 1000 genes processed
        if gene_count % 1000 == 0:
            print(f"Computed correlations for {gene_count} / {len(unique_genes)} genes.")


results = pd.DataFrame(correlations)
FILENAME = "../output/correlations/correlations_" + GENE_SELECTED + "_by_chr.csv"

# Save the results to a CSV file
results.to_csv(FILENAME, index=False)

# Display the first few rows of the results
results.head()

Computed correlations for 1000 / 18119 genes.
